In [6]:
import nibabel as nib
import numpy as np
import glob

# Liste des fichiers NIfTI à fusionner
file_list = glob.glob(r'/NAS/tupac/protocoles/PULSE/DTI/FA/*_onMNI.nii.gz')

# Initialiser une variable pour accumuler la somme des images
sum_image = None

# Lire chaque fichier et accumuler la somme
for file in file_list:
    img = nib.load(file)
    img_data = img.get_fdata()

    if sum_image is None:
        sum_image = np.zeros_like(img_data, dtype=np.float32)

    sum_image += img_data

# Calculer la moyenne
mean_image_data = sum_image / len(file_list)

# Créer une nouvelle image NIfTI pour la moyenne
mean_img = nib.Nifti1Image(mean_image_data, img.affine, img.header)

# Sauvegarder l'image moyenne
nib.save(mean_img, r'/NAS/tupac/protocoles/PULSE/DTI/FA/mean_image.nii.gz')


In [13]:
import nibabel as nib
import numpy as np
import glob

# Liste des fichiers NIfTI à fusionner
file_list = glob.glob(r'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/template_PAM50/sub-*/ses-*/sub-*_ses-M*_dwi_fa_on_PAM50.nii.gz')

# Initialiser une variable pour accumuler la somme des images
sum_image = None

# Lire chaque fichier et accumuler la somme
for file in file_list:
    img = nib.load(file)
    img_data = img.get_fdata()

    if sum_image is None:
        sum_image = np.zeros_like(img_data, dtype=np.float32)

    sum_image += img_data

# Calculer la moyenne
mean_image_data = sum_image / len(file_list)

# Créer une nouvelle image NIfTI pour la moyenne
mean_img = nib.Nifti1Image(mean_image_data, img.affine, img.header)

# Sauvegarder l'image moyenne
nib.save(mean_img, r'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/mean_image_FA_onPAM50.nii.gz')


In [10]:
file_list

[]

In [102]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline
import matplotlib.pyplot as plt

# # Lire les fichiers avec nibabel
# #images = [ for file_path in file_paths]
# # Soustraire l'image '801T2W_SPAIR2D_T2w_masklesioncervicalcord_dilated' de 'seg_cerv_rpi_labeled_eroded'
# dilated_les_M6 = nib.load('/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3_fused2/sub-012/ses-M6/sub-012_ses-M6_acq-801T2W_SPAIR2D_T2w_masklesioncervicalcord_dilated.nii.gz').get_fdata()
# eroded_les_M6 = nib.load('/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3_fused2/sub-012/ses-M6/sub-012_ses-M6_acq-801T2W_SPAIR2D_T2w_masklesioncervicalcord_eroded.nii.gz').get_fdata()
# segmented_vert_M6 = nib.load('/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/sct_4.3_fused2/sub-012/ses-M6/seg_cerv_rpi_labeled_eroded.nii.gz').get_fdata()


# # Ne garder que les labels 3 à 6 dans dilated_img
# filtered_vert_data = np.where((segmented_vert_M6 >= 3) & (segmented_vert_M6 <= 6), 1, 0)
# result_vertminusles_data = filtered_vert_data - dilated_les_M6 - dilated_les_M3

# Chemin des fichiers à récupérer
#file_pattern = r'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/*/fa_NAWM_mean.txt'
file_pattern = r'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/*/fa_M3onM6.nii.gz'

# Récupérer les fichiers avec une taille supérieure à 0 octet
valid_files = [file for file in glob.glob(file_pattern) if os.path.getsize(file) > 0]

# Afficher les fichiers récupérés
print("Fichiers valides :", valid_files)
# Créer un dictionnaire pour stocker les données
data = {}


# Afficher des informations sur chaque fichier chargé
for i, img in enumerate(valid_files[:]):    
    SUBJ = img.split('/')[8]
    # print(f"Traitement du fichier ${SUBJ} {i+1}/{len(valid_files[:])}: {img}")
    if os.path.exists(img.replace('fa_M3onM6.nii.gz','fa_M3onM6.rotated.nii.gz')) :
        DWI_quant = nib.load(img.replace('fa_M3onM6.nii.gz','fa_M3onM6.rotated.nii.gz')).get_fdata()
        print(f"Vérification des fichiers : {img.replace('fa_M3onM6.nii.gz','fa_M3onM6.rotated.nii.gz')}")
    else :
        DWI_quant = nib.load(img).get_fdata()

    # Calculer la moyenne de DWI_quant pour les voxels dans filtered_vert_data
    nawm_name = f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{SUBJ}/NAWM.nii.gz'
    lesion_name = f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{SUBJ}/mask_lesion_M6_eroded_3_6.nii.gz'
    
    if os.path.exists(nawm_name) and os.path.exists(lesion_name):
        nawm_data = nib.load(nawm_name).get_fdata()
        mean_nawm = np.mean(DWI_quant[(nawm_data == 1) & (DWI_quant != 0)])
        
        filtered_vert_data = nib.load(lesion_name).get_fdata()    
        # Calculer la moyenne de DWI_quant pour les voxels dans result_vertminusles_data
        print(np.sum(filtered_vert_data))
        if np.sum(filtered_vert_data) == 0:
            mean_lesion = 0
        else:
            mean_lesion = np.mean(DWI_quant[(filtered_vert_data == 1) & (DWI_quant != 0)])

        # Ajouter les résultats au dictionnaire
        data[SUBJ] = {'mean_nawm': mean_nawm, 'mean_lesion': mean_lesion}

# Créer un DataFrame à partir du dictionnaire
df_results = pd.DataFrame.from_dict(data, orient='index')

# Afficher le DataFrame
print(df_results)

# # Afficher l'histogramme
# plt.hist(DWI_quant[filtered_vert_data == 1], bins=50, color='blue', alpha=0.7)
# plt.title("Histogramme des valeurs de DWI_quant pour NAWM")
# plt.xlabel("Valeurs de DWI_quant")
# plt.ylabel("Fréquence")
# plt.show()

    
    


Fichiers valides : ['/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-078/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-126/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-107/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-048/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-010/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-040/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-013/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-090/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-008/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-098/fa_M3onM6.nii.gz', '/NAS/coolio/protocoles/CINOCIS/BIDS/der

/home/global/Anaconda3-2023.09/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning:

Mean of empty slice.

/home/global/Anaconda3-2023.09/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning:

invalid value encountered in scalar divide



0.0
Vérification des fichiers : /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-126/fa_M3onM6.rotated.nii.gz
0.0
0.0
Vérification des fichiers : /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-048/fa_M3onM6.rotated.nii.gz
Vérification des fichiers : /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-010/fa_M3onM6.rotated.nii.gz
251.0
Vérification des fichiers : /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-040/fa_M3onM6.rotated.nii.gz
140.0
Vérification des fichiers : /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-013/fa_M3onM6.rotated.nii.gz
153.0
Vérification des fichiers : /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-090/fa_M3onM6.rotated.nii.gz
0.0
Vérification des fichiers : /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-008/fa_M3onM6.rotated.nii.gz
70.0
Vérification des fichiers : /NAS/coolio/protocoles/CINOCIS/BIDS/derivati

In [ ]:
df_results.rename(columns={'index': 'Subject'}, inplace=True)


,mean_nawm,mean_lesion
sub-078,NaN,0.000000
sub-126,0.357825,0.000000
sub-107,0.384235,0.000000
sub-010,0.244676,0.276577
sub-040,0.285849,NaN
...,...,...
sub-038,NaN,NaN
sub-119,NaN,0.000000
sub-015,0.293026,0.000000
sub-026,NaN,NaN


In [110]:
df_results

,mean_nawm,mean_lesion
sub-078,NaN,0.000000
sub-126,0.357825,0.000000
sub-107,0.384235,0.000000
sub-010,0.244676,0.276577
sub-040,0.285849,NaN
...,...,...
sub-038,NaN,NaN
sub-119,NaN,0.000000
sub-015,0.293026,0.000000
sub-026,NaN,NaN


In [109]:
from scipy.stats import ttest_ind

import plotly.express as px

# Créer un DataFrame avec les données triées
# df_results = df_results.sort_values(by="FA_NAWM")

# Créer un graphique interactif avec Plotly
fig = px.box(
    df_results.melt(id_vars=["Subject"], value_vars=["mean_nawm", "mean_lesion"]),
    x="variable",
    y="value",
    color="variable",
    hover_data=["Subject"],
    labels={"value": "FA Values", "variable": "Group"},
    title="Boxplots interactifs de FA_NAWM et FA_lesion"
)

# Afficher le graphique
fig.show()

# # Effectuer un test t pour comparer les deux groupes
# t_stat, p_value = ttest_ind(fa_nawm_values.dropna(), fa_lesion_values.dropna(), equal_var=False)

# # Afficher les résultats du test t
# print(f"T-statistic: {t_stat}, P-value: {p_value}")

KeyError: "The following 'id_vars' are not present in the DataFrame: ['Subject']"

In [40]:
import glob
import os
import pandas as pd

# Chemin des fichiers à récupérer
file_pattern = r'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/*/fa_NAWM_mean.txt'

# Récupérer les fichiers avec une taille supérieure à 0 octet
valid_files = [file for file in glob.glob(file_pattern) if os.path.getsize(file) > 0]

# Afficher les fichiers récupérés
print("Fichiers valides :", valid_files)
# Créer un dictionnaire pour stocker les données
data = {}

# Parcourir les fichiers valides
for file in valid_files:
    # Extraire la clé "sub-..." du chemin
    key = os.path.basename(os.path.dirname(file))
    
    # Lire la valeur FA_NAWM contenue dans le fichier
    value = float(open(file, 'r').read().strip())
    
    # Ajouter la clé et la valeur au dictionnaire
    #dta[key] = value

    # Chemin du fichier fa_lesion correspondant
    lesion_file = os.path.join(os.path.dirname(file), 'fa_lesion_M6_mean.txt')
    
    # Vérifier si le fichier fa_lesion existe et a une taille > 0
    if os.path.exists(lesion_file) and os.path.getsize(lesion_file) > 0:
        lesion_value = float(open(lesion_file, 'r').read().strip())
    else:
        lesion_value = 0  # Valeur par défaut si le fichier n'existe pas ou est vide
    
    # Ajouter la valeur fa_lesion au dictionnaire
    if value > 0 and lesion_value > 0:
        data[key] = {'FA_NAWM': value, 'FA_lesion': lesion_value}


# Créer un DataFrame à partir du dictionnaire
df_FA_NAWM = pd.DataFrame.from_dict(data, orient='index').reset_index()
df_FA_NAWM.rename(columns={'index': 'Subject'}, inplace=True)

# Afficher le DataFrame
print(df_FA_NAWM)

Fichiers valides : ['/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-078/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-126/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-107/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-010/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-040/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-013/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-090/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-008/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-098/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-111/fa_NAWM_mean.txt', '/NAS/coolio/protocoles/CINOCIS/BIDS/der

In [ ]:
import nibabel as nib
import os
import pandas as pd
import glob
import numpy as np

# Initialiser une liste pour stocker les résultats
results = []
fa_files = glob.glob('/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-091/fa_M3onM6.nii.gz')
fa_files = sorted(fa_files)

# Parcourir tous les sujets possibles
for fa_path in fa_files:
    subj = os.path.basename(os.path.dirname(fa_path))
    # Pour chaque type de fichier (fa, ad, adc)
    for modality in ['fa', 'ad', 'adc']:
        if os.path.exists('/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-091/fa_M3onM6.rotated.lps.nii.gz'):
            modality_path = '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-091/fa_M3onM6.rotated.lps.nii.gz'
        else:
            modality_path = '/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-091/fa_M3onM6.lps.nii.gz'
        print(modality_path)
        if os.path.exists(modality_path):
            nawm_path = f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/NAWM.nii.gz'
            mask_lesion_path = f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/mask_lesion_M6_eroded_3_6_lps.nii.gz'
            print(f'patient {subj} - {modality}')
            if os.path.exists(nawm_path) and os.path.exists(mask_lesion_path):
                nawm_img = nib.load(nawm_path)
                mask_lesion_img = nib.load(mask_lesion_path)
                modality_img = nib.load(modality_path)

                nawm_data = nawm_img.get_fdata()
                mask_lesion_data = mask_lesion_img.get_fdata()
                modality_data = modality_img.get_fdata()

                mean_nawm = np.nanmean(modality_data[(nawm_data == 1) & (modality_data != 0)])
                mean_lesion = np.nanmean(modality_data[(mask_lesion_data == 1) & (modality_data != 0)])
                print(f'Mean {modality.upper()}_NAWM for {subj}: {mean_nawm}')
                print(f'Mean {modality.upper()}_lesion for {subj}: {mean_lesion}')
                results.append({'Subject': subj, f'{modality.upper()}_NAWM': mean_nawm, f'{modality.upper()}_lesion': mean_lesion})
                
# Créer un DataFrame à partir des résultats
df_results = pd.DataFrame(results)

# Afficher le DataFrame
print(df_results)


patient sub-091 - fa
Mean FA_NAWM for sub-091: 0.40502303011967367
Mean FA_lesion for sub-091: 0.021712585538625716
patient sub-091 - ad


/tmp/ipykernel_12277/372554087.py:34: RuntimeWarning:

Mean of empty slice



Mean AD_NAWM for sub-091: nan
Mean AD_lesion for sub-091: 0.003137735651978859
patient sub-091 - adc
Mean ADC_NAWM for sub-091: nan
Mean ADC_lesion for sub-091: 0.0021588259705617855
   Subject   FA_NAWM  FA_lesion  AD_NAWM  AD_lesion  ADC_NAWM  ADC_lesion
0  sub-091  0.405023   0.021713      NaN        NaN       NaN         NaN
1  sub-091       NaN        NaN      NaN   0.003138       NaN         NaN
2  sub-091       NaN        NaN      NaN        NaN       NaN    0.002159


In [80]:
fa_files = glob.glob('/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/*/fa_M3onM6.nii.gz')
fa_files = sorted(fa_files)

# Parcourir tous les sujets possibles
for fa_path in fa_files:
    subj = os.path.basename(os.path.dirname(fa_path))
    
    files_dir1 = glob.glob(f'/NAS/coolio/protocoles/CINOCIS/BIDS/{subj}/ses-M6/anat/*SPAIR2D_T2w*.nii.gz')
    fa_files = glob.glob(f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/fa*.nii.gz')
    adc_file = glob.glob(f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{subj}/adc*.nii.gz')
    print(f"freeview {files_dir1[0]} {fa_files[0]} {adc_file[0]}")

freeview /NAS/coolio/protocoles/CINOCIS/BIDS/sub-003/ses-M6/anat/sub-003_ses-M6_acq-1901T2W_SPAIR2D_T2w.nii.gz /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-003/fa_M3onM6.nii.gz /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-003/adc_M3onM6.nii.gz
freeview /NAS/coolio/protocoles/CINOCIS/BIDS/sub-004/ses-M6/anat/sub-004_ses-M6_acq-801T2W_SPAIR2D_T2w.nii.gz /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-004/fa_M3onM6.nii.gz /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-004/adc_M3onM6.nii.gz
freeview /NAS/coolio/protocoles/CINOCIS/BIDS/sub-005/ses-M6/anat/sub-005_ses-M6_acq-801T2W_SPAIR2D_T2w.nii.gz /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-005/fa_M3onM6.nii.gz /NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-005/adc_M3onM6.nii.gz
freeview /NAS/coolio/protocoles/CINOCIS/BIDS/sub-006/ses-M6/anat/sub-006_ses-M6_acq-1801T2W_SPAIR2D_T2w.nii.gz /NAS/cool

In [ ]:
import glob

SUBJECT = 'sub-091'
print(df_results[df_results['Subject'] == SUBJECT])


# Afficher les fichiers du premier répertoire
files_dir1 = glob.glob(f'/NAS/coolio/protocoles/CINOCIS/BIDS/{SUBJECT}/ses-M6/anat/*SPAIR2D_T2w*.nii.gz')
for file in files_dir1:
    print(file)


# Afficher les fichiers du premier répertoire
files_dir1 = glob.glob(f'/NAS/coolio/protocoles/CINOCIS/BIDS/{SUBJECT}/ses-M3/anat/*SPAIR2D_T2w*.nii.gz')
for file in files_dir1:
    print(file)

# Afficher les fichiers du deuxième répertoire
files_dir2 = glob.glob(f'/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/{SUBJECT}/*.nii.gz')
print("Fichiers dans le deuxième répertoire:")
for file in files_dir2:
    print(file)


    Subject  FA_NAWM  FA_lesion
64  sub-091  0.40518    0.67741
Fichiers dans le premier répertoire:
/NAS/coolio/protocoles/CINOCIS/BIDS/sub-091/ses-M6/anat/sub-091_ses-M6_acq-1501T2W_SPAIR2D_T2w.nii.gz
Fichiers dans le premier répertoire:
/NAS/coolio/protocoles/CINOCIS/BIDS/sub-091/ses-M3/anat/sub-091_ses-M3_acq-1501T2W_SPAIR2D_T2w_res_RPI_seg.nii.gz
/NAS/coolio/protocoles/CINOCIS/BIDS/sub-091/ses-M3/anat/sub-091_ses-M3_acq-1501T2W_SPAIR2D_T2w.nii.gz
/NAS/coolio/protocoles/CINOCIS/BIDS/sub-091/ses-M3/anat/sub-091_ses-M3_acq-1501T2W_SPAIR2D_T2w_lesionseg_recM3onM6_warped.nii.gz
/NAS/coolio/protocoles/CINOCIS/BIDS/sub-091/ses-M3/anat/sub-091_ses-M3_acq-1501T2W_SPAIR2D_T2w_RPI_seg.nii.gz
Fichiers dans le deuxième répertoire:
/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-091/fa_M3onM6.nii.gz
/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-091/lesion_M3_dilated.nii.gz
/NAS/coolio/protocoles/CINOCIS/BIDS/derivatives_v2.0/predictiveFA/sub-091/ad_

In [36]:
df_results.columns

Index(['Subject', 'FA_NAWM', 'FA_lesion', 'AD_NAWM', 'AD_lesion', 'ADC_NAWM',
       'ADC_lesion'],
      dtype='object')

In [41]:
from scipy.stats import ttest_ind

import plotly.express as px

# Créer un DataFrame avec les données triées
# df_FA_NAWM_sorted = df_results.sort_values(by="FA_NAWM")

# Créer un graphique interactif avec Plotly
fig = px.box(
    df_results.melt(id_vars=["Subject"], value_vars=["FA_NAWM", "FA_lesion"]),
    x="variable",
    y="value",
    color="variable",
    hover_data=["Subject"],
    labels={"value": "FA Values", "variable": "Group"},
    title="Boxplots interactifs de FA_NAWM et FA_lesion"
)

# Afficher le graphique
fig.show()

# Effectuer un test t pour comparer les deux groupes
t_stat, p_value = ttest_ind(df_results["FA_NAWM"].dropna(), df_results["FA_lesion"].dropna(), equal_var=False)

# Afficher les résultats du test t
print(f"T-statistic: {t_stat}, P-value: {p_value}")

T-statistic: -1.928526294387387, P-value: 0.06286344814171646


In [42]:
df_FA_NAWM_sorted_nolesionna = df_FA_NAWM_sorted.dropna(subset=['FA_lesion'])
lowest_fa_lesion_subjects = df_FA_NAWM_sorted_nolesionna.nsmallest(5, 'FA_lesion')
print(lowest_fa_lesion_subjects[['Subject', 'FA_lesion']])

    Subject  FA_lesion
64  sub-091   0.021713
70  sub-099   0.149393
8   sub-013   0.174345
3   sub-008   0.220028
5   sub-010   0.277457
